In [3]:
# With this operation, a comprehensive language model is downloaded from the 'spacy' library. There are three versions to it:
# - Large
# - Medium
# - Small
# In this case, we are using the smaller model, which is around 30 MB.

!python -m spacy download en_core_web_sm
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!uv pip install openai==0.27.7

C:\Users\koste\Desktop\UW\Arbitraz\.venv\Scripts\python.exe: No module named pip
Using Python 3.12.12 environment at: C:\Users\koste\Desktop\UW\Arbitraz\.venv
Resolved 17 packages in 1.65s
Prepared 2 packages in 722ms
Installed 8 packages in 271ms
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.2
 + aiosignal==1.4.0
 + frozenlist==1.8.0
 + multidict==6.7.0
 + openai==0.27.7
 + propcache==0.4.1
 + yarl==1.22.0


In [4]:
!uv pip install numpy==1.26.4

Using Python 3.12.12 environment at: C:\Users\koste\Desktop\UW\Arbitraz\.venv
Audited 1 package in 51ms


In [5]:
!uv pip install spacy==3.7.1

Using Python 3.12.12 environment at: C:\Users\koste\Desktop\UW\Arbitraz\.venv
Audited 1 package in 93ms


In [6]:
!uv pip install -U sentence-transformers rank_bm25

Using Python 3.12.12 environment at: C:\Users\koste\Desktop\UW\Arbitraz\.venv
Resolved 32 packages in 1.97s
Prepared 16 packages in 1m 37s
Uninstalled 3 packages in 870ms
Installed 16 packages in 1m 24s
 - certifi==2025.10.5
 + certifi==2025.11.12
 + filelock==3.20.0
 + fsspec==2025.10.0
 + huggingface-hub==0.36.0
 + mpmath==1.3.0
 + networkx==3.5
 - numpy==1.26.4
 + numpy==2.3.5
 + rank-bm25==0.2.2
 + regex==2025.11.3
 + safetensors==0.6.2
 - scipy==1.11.4
 + scipy==1.16.3
 + sentence-transformers==5.1.2
 + sympy==1.14.0
 + tokenizers==0.22.1
 + torch==2.9.1
 + transformers==4.57.1


In [7]:

# Wczytaj plik zawartość pliku Neurips.csv z kubełka COS i sprawdź co jest w środku

storage_metadata = {
    'IAM_SERVICE_ID': 'iam-ServiceId-38022511-be04-4668-b921-4ce2f2831b4b',
    'IBM_API_KEY_ID': 'ZGFcYW5bFVwSxRZ79nsgthmMnK1lLGfrC10htZE3tLsY',
    'ENDPOINT': 'https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/identity/token',
    'BUCKET': '24mas25z-donotdelete-pr-2vi4c2g82muyx5',
    'FILE': 'Neurips.csv'
}



In [ ]:
import json
import pandas as pd
import time
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest
import nltk
import numpy as np
from tqdm import tqdm
import openai

import matplotlib.pyplot as plt
import seaborn as sns

### <span style="color:violet">Pracujemy z danymi:</span>

In [14]:
# Wczytaj plik zawartość pliku Neurips.csv z kubełka COS

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self):
    yield self.read()

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZGFcYW5bFVwSxRZ79nsgthmMnK1lLGfrC10htZE3tLsY',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud')

bucket = '24mas25z-donotdelete-pr-2vi4c2g82muyx5'
object_key = 'Neurips.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"):
    body.__iter__ = types.MethodType( __iter__, body )

df_in = pd.read_csv(body)
df_in.head(10)

ConnectTimeoutError: Connect timeout on endpoint URL: "https://s3.direct.eu-gb.cloud-object-storage.appdomain.cloud/24mas25z-donotdelete-pr-2vi4c2g82muyx5/Neurips.csv"

In [ ]:
# Proszę zrobić przegląd danych
df.head(5)

In [ ]:
# Szybki przegląd danych
print(df.nunique())

In [ ]:
# Sprawdź czy dane sa kompletne
missing_values = df.isnull().sum()
print(missing_values)

In [ ]:
# Sprawdź czy występuja duplikaty

In [ ]:
# Podziel publikacje latami
# Określ liczebność każdej z grup
# Zwizualizuj na wykresie słupkowym
# Wyświetl rysunek z legenda

In [ ]:
# Polacz wszystkie tytuly w jeden string "all_titles"
all_titles = ' '.join(df['Title'].dropna())

In [ ]:
# Wstępne przetwarzanie tekstu
import re
titles_cleaned = re.sub(r'[^\w\s]','', all_titles).lower().split()

In [ ]:
nltk.download('stopwords')

In [ ]:
# Usuń slowa typu 'stopwords' korzystajac z biblioteki nltk.corpus
# ... zacznij od nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
titles_cleaned = [word for word in titles_cleaned if word not in stop_words]

In [ ]:
# Znajdź TOP-20 najczęściej używanych slów

from collections import Counter
word_counts = Counter(titles_cleaned)
common_words = word_counts.most_common(20)

In [ ]:
# pobierz z df zawartośc kolumn 'word' i 'count' i przygotuj wykres slupkowy wraz z legenda
words_df = pd.DataFrame(common_words, columns=['word', 'count'])

...........

In [ ]:
# Znajdźmy slowa kluczowe
def extract_keywords(titles):
    keywords = []
    for title in titles:
        words = re.sub(r'[^\w\s]','', str(title)).lower().split()
        words = [word for word in words if word not in stop_words]
        keywords.extend(words)
    return keywords

In [ ]:
# Dodajmy kolumnę 'keywords' do ramki danych z użyciem funkcji extract_keywords
.......

In [ ]:
df.head(5)

In [ ]:
# Stwórz ramkę keywords_per_year rozpakowujac kolekcje 'keywords' zawarte w komórkach ramki df w oparciu o klucz 'Year'


In [ ]:
keywords_per_year[:10]

In [ ]:
# Wybierzmy konkretne slowa:
topics = ['neural', 'learning', 'network', 'deep', 'reinforcement']

In [ ]:
# higiena .....
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
# Przygotuj wykres częstości występowania konkretnych tematów badawczych na przestrzeni lat



### <b><span style="color:green">Wyszukiwanie semantyczne</span></b> na zbiorze abstractów:

In [ ]:
# Zapisz ramkę df we formacie pkl - będzie nam to potrzebne do zachowania wyników pośrednich

In [ ]:
import io

In [ ]:
# Zapisz DataFrame do bufora w pamięci
buffer = io.BytesIO()
df.to_pickle(buffer)
buffer.seek(0)  # Przewiń na początek bufora

# Zapisz używając wslib
wslib.save_data('df_pickle.....pkl', buffer.read())

In [ ]:
# Odzyskaj ramkę z pliku pkl za pomoca metody .read_pickle

In [ ]:
import os, types
import pandas as pd

In [ ]:
df_with_embedding = pd.read_pickle(.........)

In [ ]:
df_with_embedding.head(5)

In [ ]:
df = df_with_embedding.copy()

In [ ]:
# "Połącz kolumny Title i Abstract w jedną kolumnę Combined, dodając etykiety 
# 'Title:' i 'Abstract:' oraz usuwając nadmiarowe spacje."


df["Combined"] = (
    "Title: " + df.Title.str.strip()+"; Abstract: " + df.Abstract.str.strip()
)

In [ ]:
# Skopiuj ramkę danych, a następnie w kolumnie Combined usuń znaki specjalne i zamień wszystkie litery na małe.

import re

df_combined = df.copy()

df_combined['Combined'] = df_combined['Combined'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',str(x)))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['Combined']= df_combined['Combined'].apply(lambda x: lower_case(x))

#### <span style="color:#27F531;">Przetwarzanie równoległe?</span>

In [ ]:
import torch

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("gpu")
    print("using GPU")
else:
    device = torch.device("cpu")
    print("using CPU")

In [ ]:
# Pakietowe czary mary ...
!pip install numpy==1.26.4 pandas torch==2.3.1 transformers==4.44.0

In [ ]:
# a teraz zrestartuj kernel i sprawdź:
import numpy as np
import torch
import transformers
print("NumPy:", np.__version__, "✓")
print("PyTorch:", torch.__version__, "✓")
print("Transformers:", transformers.__version__, "✓")

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os

embedder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
df_combined.head()

In [ ]:
# Pobierz próbkę w postaci pierwszych 10-ciu wierszy
sample = df_combined[:10]

In [ ]:
import time

In [ ]:
# Wersja CPU
embedder =  embedder.to('cpu')

startTime = time.time()
# Utwórz kolumnę o nazwie "embedding”, w której kolumna "Combined” zostanie przekształcona na osadzenia przez model.
sample["embedding"] = sample.Combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
# Wersja GPU.

embedder =  embedder.to('mps')
startTime = time.time()

sample["embedding"] = sample.Combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


In [ ]:
sample

In [ ]:
# Zapisz DataFrame do bufora w pamięci
buffer = io.BytesIO()
sample.to_pickle(buffer)
buffer.seek(0)  # Przewiń na początek bufora

# Zapisz używając wslib
wslib.save_data('sample.pkl', buffer.read())

In [ ]:
df_with_embedding = pd.read_pickle(wslib.load_data('sample.pkl')) #to load df123.pkl back to the dataframe df

### Query!

In [ ]:
query = 'vanishing gradients'

# Embed the previous query.
query_embedding = embedder.encode(query,show_progress_bar=True)

In [ ]:
# Co mamy w kolumnie Combined?
df_with_embedding.Combined[0]

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
# Utwórz listę osadzeń na podstawie zawartości kolumny "Combined”.
corpus_embeddings = embedder.encode(df_with_embedding.Combined,show_progress_bar=True)



# Utwórz kolumnę o nazwie "similarity”, wyświetlającą podobieństwo cosinusowe między osadzonym zapytaniem 
# a osadzeniem zawartości zapytania `query_embedding`.
df_with_embedding['similarity']=cosine_similarity(corpus_embeddings, query_embedding)

In [ ]:
df_with_embedding

### <span style="color:red">A teraz wszystkie dane (ostrożnie!):</span>

In [ ]:
import pandas as pd

In [ ]:
embedder =  embedder.to('mps')

startTime = time.time()


df_combined["embedding"] = df_combined.Combined.apply(lambda x: embedder.encode(x))

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

#### <span style="color:red">Nowy punkt startowy:</span>

In [ ]:
df_combined = pd.read_pickle(...) 

In [ ]:
df_combined.shape

In [ ]:
df=df_combined.copy()

In [ ]:
# przeszukaj streszczenia
def search_abstracts(df, query, n=5, pprint=True):

    # Embed your search query.
    query_embedding = embedder.encode(query,show_progress_bar=True)

    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding.reshape(768,-1))) #similarity against each doc

    # A teraz posortuj wartości według podobieństwa i wybierz najbardziej podobny dokument.
    results = (
        df.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [ ]:
# Wersja podkręcona
def search_abstracts_vectorized(df, query, n=5, pprint=True):

    # Embed your search query.
    query_embedding = embedder.encode(query,show_progress_bar=True)

    embedding_matrix = np.vstack(df.embedding.values)
    # Oblicz podobieństwa jednym razem (dużo szybsze!)
    similarities = cosine_similarity(embedding_matrix, query_embedding.reshape(768, -1))
    df["similarity"] = similarities.flatten()
    
    results = (
        df.sort_values("similarity", ascending=False) # re-rank
        .head(n))

    return results

In [ ]:
query = 'vanishing gradients'

In [ ]:
results = search_abstracts(df,query,3)

In [ ]:
results

In [ ]:
results_vectorized = search_abstracts_vectorized(df,query,3)[['Title','Authors']]

In [ ]:
results_vectorized

### <span style="color:blue">A w następnych rozdzialach ... ekspresowe API ... </span>